In [1]:
import pandas as pd
import os
import numpy as np
from spellchecker import SpellChecker
import re

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [2]:
# Get the data into a pandas dataframe
train_file_path = "/Users/yigitatay/Desktop/SentNLP/data/train.csv"
df = pd.read_csv(train_file_path)
test_file_path = "/Users/yigitatay/Desktop/SentNLP/data/test.csv"
df_test = pd.read_csv(test_file_path)

In [3]:
# separate labels and data
texts = df["text"]
selected_texts = df["selected_text"]
sentiments = df["sentiment"]
# a list to hold text, sentiment dictionaries
train_list = []
# a list to hold the labels
label_list = []
only_texts = []

for text, data, label in zip(texts, sentiments, selected_texts):
    dict_to_add = dict()
    dict_to_add[text] = data
    train_list.append(dict_to_add)
    label_list.append(label)
    list_to_add = str(text).split()
    only_texts.append(list_to_add)
    
# separate labels and data
texts = df_test["text"]
sentiments = df_test["sentiment"]
# a list to hold text, sentiment dictionaries
test_list = []

for text, data in zip(texts, sentiments):
    dict_to_add = dict()
    dict_to_add[text] = data
    test_list.append(dict_to_add)
    list_to_add = text.split()
    only_texts.append(list_to_add)

In [4]:
# get strings with all text, positive, neutral and negative test
# for non-deep-learning analysis
all_text_train = ""
all_text_test = ""
all_text = ""
all_selected_text = ""
positive_text = ""
positive_selected_text = ""
negative_text = ""
negative_selected_text = ""
neutral_text = ""
neutral_selected_text = ""
for item, label in zip(train_list, label_list):
    all_selected_text += (" " + str(label))
    for key in item:
        all_text_train += (" " + str(key))
        if(item[key] == "positive"):
            positive_text += (" " + str(key))
            positive_selected_text += (" " + str(label))
        elif(item[key] == "negative"):
            negative_text += (" " + str(key))
            negative_selected_text += (" " + str(label))
        else:
            neutral_text += (" " + str(key))
            neutral_selected_text += (" " + str(label))
            
for item in test_list:
    for key in item:
        all_text_test += (" " + str(key))
        if(item[key] == "positive"):
            positive_text += (" " + str(key))
            positive_selected_text += (" " + str(label))
        elif(item[key] == "negative"):
            negative_text += (" " + str(key))
            negative_selected_text += (" " + str(label))
        else:
            neutral_text += (" " + str(key))
            neutral_selected_text += (" " + str(label))
all_text = all_text_train + all_text_test

In [5]:
all_words_list = all_text.split()

In [6]:
spell = SpellChecker()
word2vec_train = []
add_this_list = []
print(len(only_texts))
for i, lst in enumerate(only_texts):
    for word in lst:
        if word[0:4] == "http":
            continue
        if "****" in word:
            word = word.replace('****', 'censored')
        str1 = ''
        str2 = ''
        switch = False
        for char in word:
            if char.isalpha() or char=="`" or char=="-":
                if switch:
                    str2 = str2 + char
                else:
                    str1 = str1 + char
            else:
                switch = True
        count = 1
        tempChar = ''
        newStr = ''
        for char in str1:
            if char == tempChar:
                count += 1
            tempChar = char
            if count < 3:
                newStr += char
            else:
                newStr = newStr[:-1]
                continue
        count = 1
        tempChar = ''
        newStr2 = ''
        for char in str2:
            if char == tempChar:
                count += 1
            tempChar = char
            if count < 3:
                newStr2 += char
            else:
                newStr2 = newStr[:-1]
                continue
        if newStr != "" and newStr != "`" and newStr != "-":
#             add_this_list.append(spell.correction(newStr.lower()))
            add_this_list.append(newStr.lower())
        if newStr2 != "" and newStr2 != "`" and newStr2 != "-":
#             add_this_list.append(spell.correction(newStr2.lower()))
            add_this_list.append(newStr2.lower())
    word2vec_train.append(add_this_list)
    add_this_list = []

31015


In [7]:
model = Word2Vec(common_texts, size=40, window=5, min_count=1, workers=4)
model.train(word2vec_train, total_examples=len(word2vec_train), epochs=30)
model.save("word2vec_20.model")

In [8]:
word_vectors = model.wv
print(word_vectors["bad"])
word_vectors.most_similar(positive="france")

KeyError: "word 'bad' not in vocabulary"

In [11]:
new_model = Word2Vec.load("word2vec_checkpoints/word2vec_80.model")

In [12]:
word_vecs = new_model.wv
print(word_vecs["bad"])
word_vecs.most_similar(positive="france")

[ 2.7999861e+00  3.4378700e+00  1.3343029e-01 -1.7258933e+00
 -2.1933072e+00 -1.2163146e+00 -4.2054425e-03 -4.7087115e-01
 -2.8000445e+00 -3.0226305e+00 -1.1610838e+00  3.0894806e+00
  1.4331795e-01 -3.2117149e-01 -3.2210478e-01 -4.9234846e-01
 -1.2134603e+00  2.8057344e+00  1.3026572e+00 -3.3423722e-01
  2.5895295e+00 -3.6200345e-01 -3.0259647e+00  1.0741885e+00
  3.7096077e-01  3.8122158e+00 -3.8140813e-01 -5.0311503e+00
  1.2573861e+00  1.7053988e-02 -4.8959839e-01  8.0294943e-01
  6.2059212e-01  1.2024442e+00  1.9188854e+00  1.6187038e+00
  4.4396541e-01 -1.7715130e+00  1.4294227e+00  3.8013194e+00
 -1.3466917e+00 -1.5571499e+00  2.0741105e+00 -2.5135386e-01
  6.2788832e-01  2.3422976e+00 -3.5756688e+00  2.7249253e+00
  1.3126025e+00  6.3044471e-01  1.0292249e+00  5.0902641e-01
 -2.8560450e+00 -2.5765204e+00 -2.1603413e+00 -1.6816295e+00
  4.6173251e-01  1.5901607e-01 -8.7663877e-01  2.0076752e+00
  8.6416072e-01  5.6010729e-01  6.6222802e-02 -8.6823165e-01
  2.3556492e+00 -1.23968

[('dallas', 0.6668902635574341),
 ('london', 0.5933231115341187),
 ('paso', 0.5720329284667969),
 ('vegas', 0.5719572305679321),
 ('francisco', 0.5640627145767212),
 ('california', 0.5627366304397583),
 ('tangerine', 0.5569376945495605),
 ('deg', 0.5531080961227417),
 ('delay', 0.5504189729690552),
 ('garden', 0.5501287579536438)]